In [1]:
## For debbugging
%load_ext autoreload

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
## for debugging:
%aimport mlp.keyword
%aimport mlp.tools
%autoreload 1
from mlp.keyword import *

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline

import spacy, textacy, platform
print(
    f'Textacy: \t{textacy.__version__:}'+\
    f'\nspaCy: \t{spacy.__version__:>13}'+\
    f'\nPython: \t{platform.python_version()}'
    )

Textacy: 	0.4.1
spaCy: 	        1.9.0
Python: 	3.6.3


In [4]:
data_directory = os.path.join(module_path,'./data', 'gs_data')
nl_col = {
    "Description": 5,
    "Resolution": 6
}

#TODO
# meta_col = {

# }

kex_gs = KeywordExtractor('Maintenance_All_Clean.xlsx',
                       nlp_cols=nl_col,
#                       meta_cols = meta_col,
                       special_replace={' -': '; '},
                       wdir=data_directory,
                       keep_temp_files=False
                      )

450it [00:00, 4498.00it/s]

making new file...


3435it [00:00, 5661.41it/s]


Replacing NaN
Replacing NaN
Replacing NaN
Replacing NaN


In [5]:
# what we extracted...we'll be using 'RawText'
kex_gs.df.head()

,Description,Resolution,RawText
0,Broken bar feeder chain,Repaired,break bar feeder chain repair
1,No power,Replaced pin in pendant and powered machine; P...,no power replace pin in pendant and power mach...
2,Smartscope harness broken,Parts ordered / Tech repaired,smartscope harness break part order tech repair
3,Check / Charge Accumulators,Where OK,check charge accumulator where ok
4,Hyd leak at saw atachment,Replaced seal in saw attachment but still leak...,hyd leak at saw atachment replace seal in saw ...


In [6]:
# use this to make a properly formatted thesaurus file for human-annotation
# kex_gs.gen_vocab('new_tag_vocab.csv', notes=True)  # rename to match e.g. hvac_vocab.csv


In [21]:
# "fit" a thesaurus to the data
kex_gs.fit(vocab='tag_vocab.csv')
kex_gs.vocab.head(n=7)

,NE,alias,note
token,,,
replace,S,replace,NaN
unit,I,unit,NaN
motor,I,motor,NaN
spindle,I,spindle,NaN
leak,P,leak,NaN
valve,I,valve,NaN
replaced,S,replace,NaN


In [16]:
# tag the data, saving to a new DF (instead of internally)

df_pred = kex_gs.transform(save=False)
df_pred.head()

100%|██████████| 3431/3431 [00:13<00:00, 246.19it/s]


,RawText,Items,Problem,Solution,eXcess,Redundant,Unknown
0,break bar feeder chain repair,"bar, chain, feeder_chain, bar_feeder_chain, ba...",broken,repair,,,
1,no power replace pin in pendant and power mach...,"cable, pin, machine, pendant","short, power",replace,possible,replace pin,
2,smartscope harness break part order tech repair,person,broken,"order, repair",,,"harness, smartscope"
3,check charge accumulator where ok,accumulator,,"check, charge",ok,,
4,hyd leak at saw atachment replace seal in saw ...,"attachment, saw_attachment, hydraulic, saw, seal",leak,replace,see,,"reapir, ml"


In [17]:
# tag-complete work-orders?
print(df_pred[df_pred.Unknown ==''].shape[0])
df_pred[df_pred.Unknown ==''].head()

1134


,RawText,Items,Problem,Solution,eXcess,Redundant,Unknown
0,break bar feeder chain repair,"bar, chain, feeder_chain, bar_feeder_chain, ba...",broken,repair,,,
1,no power replace pin in pendant and power mach...,"cable, pin, machine, pendant","short, power",replace,possible,replace pin,
3,check charge accumulator where ok,accumulator,,"check, charge",ok,,
5,cs1008 setup change over from arc1004 complete...,"thread_unit, thread, unit",,"complete, change, setup, wire",,,
7,check and charge accumulator check and charge,accumulator,,"charge, check",,check and charge,


In [18]:
# tag-empty? 
print(df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].shape[0])
df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].head()

3


,RawText,Items,Problem,Solution,eXcess,Redundant,Unknown
631,unload automation not return,,,,,,"unload, automation, return"
2562,camshaft standstill gary,,,,,,camshaft
3185,disti water empty water fill,,,,,,"fill, disti, water"


## HVAC Data

In [12]:
data_directory = os.path.join(module_path,'./data', 'hvac_data')

nl_col = {
    "Description": 5,
    "Resolution": 248
}

meta_col = {
    "AssetID": 6,
    "Location": 7,
    "Assigned": 10,
    "Reported": 43,
    "SolveDate": 11,
    "ReportDate": 44
}

kex_hvac = KeywordExtractor('hvac_raw.xlsx',
                       nlp_cols=nl_col,
                       meta_cols = meta_col,
                       special_replace={'action taken:': '',
                                        ' -': '; ',
                                        '- ': '; ',
                                        'too hot': 'too_hot',
                                        'to hot': 'too_hot',
                                        'too cold': 'too_cold',
                                        'to cold': 'too_cold'},
                       wdir=data_directory,
                       keep_temp_files=False
                      )

In [13]:
# what we extracted...we'll be using 'RawText'
kex_hvac.df.head()

,Description,AssetID,Location,Assigned,SolveDate,Reported,ReportDate,Resolution,RawText
0,CHARGE ALL M/S AND PM'S TO THIS ORDER,NaN,GAITHER,FEDERLIN,2009-04-02 14:15:38,LEM801,2009-02-12 14:27:33,"for site , other that AML Action Taken:Complete.",charge all m s and pm be to this order for sit...
1,CHARGE ALL M/S AND PM'S TO THIS ORDER FOR THE AML,NaN,215-1,FEDERLIN,2009-04-02 14:15:39,LEM801,2009-02-12 14:44:55,Action Taken: Time charged to cover PM's as ne...,charge all m s and pm be to this order for the...
2,"ADJUST,REPAI OR REPLACE FREEZE STAT",301 HV-4,301-B184,FEDERLIN,2009-04-02 14:15:41,HWRJR,2009-02-15 23:25:01,Action Taken:Reset POC Ed,adjust repai or replace freeze stat reset poc ed
3,"RESET,ADJUST OR REPAIR FREEZE STAT",304 ACU-1,304-200,FEDERLIN,2009-04-02 14:15:42,HWRJR,2009-02-17 04:20:08,Action Taken: restart POC Ed,reset adjust or repair freeze stat restart poc ed
4,"RSET,ADJUST OR REPAIR FREEZE STAT",220 ACU-8,220-400,FEDERLIN,2009-06-25 08:17:33,HWRJR,2009-02-17 04:22:38,reset freezestat,rset adjust or repair freeze stat reset freeze...


In [14]:
# use this to make a properly formatted thesaurus file for human-annotation
kex_hvac.gen_vocab('tag_vocab.csv', notes=True)  # rename to match e.g. hvac_vocab.csv


In [15]:
# "fit" a thesaurus to the data
kex_hvac.fit(vocab='hvac_vocab.csv')
kex_hvac.vocab.head()

,NE,alias,note
token,,,
room,I,room,NaN
replace,S,replace,NaN
find,S,find,NaN
check,S,check,NaN
action_take,X,action_take,NaN


In [16]:
# tag the data, saving to a new DF (instead of internally)
df_pred = kex_hvac.transform(save=False)
df_pred.head()

100%|██████████| 16986/16986 [01:18<00:00, 217.54it/s]


,RawText,Items,Problem,Solution,eXcess,Redundant,UK_tok
0,charge all m s and pm be to this order for sit...,site,,"preventative_maintenance, order, complete",s,,"m, charge all m, aml, charge"
1,charge all m s and pm be to this order for the...,,request,"order, preventative_maintenance",s,,"time, 's, m, charge all m, aml, charge"
2,adjust repai or replace freeze stat reset poc ed,"poc, freezestat, thermostat",,"reset, replace, adjust",,,"repai, ed"
3,reset adjust or repair freeze stat restart poc ed,"poc, freezestat, thermostat",,"reset, restart, repair, adjust",,,ed
4,rset adjust or repair freeze stat reset freeze...,"freezestat, thermostat",,"reset, repair, adjust",,,


In [17]:
# tag-complete work-orders?
print(df_pred[df_pred.UK_tok ==''].shape[0])
df_pred[df_pred.UK_tok ==''].head()

1049


,RawText,Items,Problem,Solution,eXcess,Redundant,UK_tok
4,rset adjust or repair freeze stat reset freeze...,"freezestat, thermostat",,"reset, repair, adjust",,,
15,too_hot,,too_hot,,,,
18,too_hot,,too_hot,,,,
23,too_cold,,too_cold,,,,
26,remove motor pulley and install on new motor s...,"pulley, motor, belt, install, switch",old,remove,new,,


In [18]:
# tag-empty? 
print(df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].shape[0])
df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].head()

21


,RawText,Items,Problem,Solution,eXcess,Redundant,UK_tok
618,b349-mod-345b b367-mod-363b a368-mod-362b,,,,,,
877,humidty too high b331,,,,,,"humidty too high, b331, humidty, high"
1642,ice machine not work a-100,,,,,,"machine, a-100, ice, ice machine"
1765,humidty real high b217,,,,,,"real, b217, humidty, high"
1807,humidty too high rm f-013,,,,,,"humidty too high, humidty, high"


## PSU Data, no tagged keywords (yet)

In [20]:
data_directory = os.path.join(module_path, './data', 'psu_data')
nl_col = {
    "ProblemDesc": 11,
    "Prob_cause": 12,
    "RepRes": 19
}

#TODO
# meta_col = {

# }

kex_psu = KeywordExtractor('psu_data.xlsx',
                       nlp_cols=nl_col,
#                       meta_cols = meta_col,
#                        special_replace={' -': '; '},
                       wdir=data_directory,
                       keep_temp_files=False
                      )

452it [00:00, 4513.56it/s]

making new file...


8887it [00:01, 5858.50it/s]


Replacing NaN
Replacing NaN
Replacing NaN


In [21]:
kex_psu.df.head()

,ProblemDesc,Prob_cause,RepRes,RawText
0,Vacuum pump is leaking oil.,add 500 ml fomblin,Replaced Vacuum Pump,vacuum pump be leak oil add 500 ml fomblin rep...
1,Airflow readings range from 50 FPM to 85FPM. ...,add 500 ml fomblin,Adjust Airspeed,airflow reading range from 50 fpm to 85fpm may...
2,Bottom shelf Connection #4 (marked) Wrap conne...,adjust printhead pressure screws,,bottom shelf connection 4 mark wrap connection...
3,Camera on autosaw needs cleaned. Having proble...,verify machine operation and perform leak test,,camera on autosaw need clean have problem pick...
4,pump at encapsulater is low on oil,Mims wasn't hooked up at time for calibration ...,Added Oil,pump at encapsulater be low on oil mim be not ...


In [22]:
kex_psu.gen_vocab('psu_vocab.csv', notes=True)

## Automotive data, no tagged keywords (yet)

In [23]:
data_directory = os.path.join(module_path, './data', 'auto_data')

nl_col = {
    "DESCRIPTION": 5
}

#TODO
# meta_col = {

# }

kex_auto = KeywordExtractor('auto_raw.xlsx',
                       nlp_cols=nl_col,
#                       meta_cols = meta_col,
#                       special_replace={'Action Taken:': ''},
                       wdir=data_directory,
                       keep_temp_files=False
                      )

1103it [00:00, 11024.02it/s]

making new file...


12410it [00:01, 7947.09it/s]

Replacing NaN


21281it [00:02, 7890.76it/s]

...

31218it [00:04, 6366.12it/s]

Replacing NaN
Replacing NaN
Replacing NaN


38756it [00:05, 7307.12it/s]

Replacing NaN


67217it [00:08, 7614.86it/s]


In [24]:
# use this to make a properly formatted thesaurus file for human-annotation
kex_auto.gen_vocab('auto_vocab.csv', notes=True)  # rename to match e.g. hvac_vocab.csv


## Mining Data, no vocab (yet)

In [25]:
data_directory = os.path.join(module_path, './data', 'mine_data')

nl_col = {
    "OriginalShorttext": 2
}

meta_col = {
    "BscStartDate":0,
    "Asset":1,
    "PMType":3,
    "Cost":4
}

kex_mine = KeywordExtractor('mine_data.csv',
                       nlp_cols=nl_col,
                      meta_cols = meta_col,
#                       special_replace={'Action Taken:': ''},
                       wdir=data_directory,
                       keep_temp_files=False
                      )

Unsupported format, or corrupt file: Expected BOF record; found b'"BscStar' ... did not find .xlsx file, attempting .csv ...


In [26]:
# use this to make a properly formatted thesaurus file for human-annotation
# kex_mine.gen_vocab('mine_vocab.csv', notes=True)  # rename to match e.g. hvac_vocab.csv


In [37]:
# "fit" a thesaurus to the data
kex_mine.fit(vocab='mine_vocab.csv')
kex_mine.vocab.head()

,NE,alias,note
token,,,
replace,S,replace,NaN
leak,P,leak,NaN
repair,S,repair,NaN
bucket,I,bucket,NaN
engine,I,engine,NaN


In [38]:
# tag the data, saving to a new DF (instead of internally)
df_pred = kex_mine.transform(save=False)
df_pred.head()

100%|██████████| 5485/5485 [00:12<00:00, 427.38it/s]


,RawText,Items,Problem,Solution,eXcess,Redundant,UK_tok
0,bucket will not open,bucket,,,,,open
1,l h bucket cyl leaking,"bucket, left_hand, h_bucket, cylinder, bucket_...",leak,,,,"l, h"
2,swap bucket,bucket,,,,,swap
3,fit bucket tooth,"bucket, tooth, bucket_tooth",,,,,fit
4,refit bucket tooth,"bucket, tooth, bucket_tooth",,refit,,,


In [39]:
# tag-complete work-orders?
print(df_pred[df_pred.UK_tok ==''].shape[0])
df_pred[df_pred.UK_tok ==''].head()

1338


,RawText,Items,Problem,Solution,eXcess,Redundant,UK_tok
4,refit bucket tooth,"bucket, tooth, bucket_tooth",,refit,,,
5,repair loose bucket cylinder,"bucket_cylinder, bucket, cylinder",loose,repair,,,
8,replace lip,lip,,replace,,,
10,replace bucket tooth,"bucket, bucket_tooth, tooth",,replace,,,
15,oil leak on bucket,"bucket, oil",leak,,,,


In [36]:
# tag-empty? 
print(df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].shape[0])
df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].head()

407


,RawText,Items,Problem,Solution,eXcess,Redundant,UK_tok
9,rebuild lip 3,,,,,,"lip, rebuild, 3"
41,rebuild spare lip get,,,,,,"lip, rebuild, spare"
49,air con not get cold,,,,,,"con, cold, air con, air"
94,get suspention rope socket,,,,,,"socket, rope"
104,mining ~ get,,,,,,"~, mining"
